In [1]:
import ray

In [2]:
ray.init(num_cpus=4)

2023-07-03 17:57:11,105	INFO worker.py:1636 -- Started a local Ray instance.


Python version:,3.9.16
Ray version:,2.5.0


In [3]:
# ray.shutdown()

In [1]:
import pandas as pd
%cd ..

import logging
from bds.meel import approx_mc2, approx_mc2_core
from bds.bb import get_ground_truth_count
from bds.rule import Rule
from bds.utils import bin_random, randints
from logzero import logger

import numpy as np
from contexttimer import Timer

logger.setLevel(logging.WARN)

num_pts = 1000

ub = 0.8
lmbd = .1

delta = .8
eps = .8

show_progres = True
rand_seed = 1234

/u/50/ciaperm1/unix/sampling-rashomon-decision-set-code


In [5]:
n_reps = 1
num_rules_list = list(reversed([ 50,]))
# num_rules_list = list(reversed([200]))


np.random.seed(rand_seed)
res_rows = []
for _ in range(n_reps):
    for num_rules in num_rules_list:
        rule_random_seeds = randints(num_rules)
        rules = [
            Rule.random(i+1, num_pts, random_seed=rule_random_seeds[i])
            for i in range(num_rules)
        ]
        y = bin_random(num_pts)

        with Timer() as cbb_timer:
            test_cnt = approx_mc2(
                rules, y, lmbd=lmbd, ub=ub,
                delta=delta, eps=eps, rand_seed=rand_seed, show_progress=show_progres,
                parallel=False, log_level=logging.WARN
            )
        test_elapsed = cbb_timer.elapsed
        
        with Timer() as bb_timer:
            ref_count = get_ground_truth_count(rules, y, lmbd, ub)
        ref_elapsed = bb_timer.elapsed        

        res_rows.append((num_rules, test_elapsed, ref_elapsed, test_cnt, ref_count))

  0%|                                                    | 0/32 [00:00<?, ?it/s]

rand_seed: 822569775


  6%|██▊                                         | 2/32 [00:00<00:06,  4.68it/s]

time_cost_info: 
|1|0.018664908973732963|
|2|0.005709743010811508|
|3|0.008809374994598329|
|4|0.013108623999869451|
|8|0.05641009900136851|
|6|0.05971185499220155|
|7|0.06463259100564755|
rand_seed: 2137449171
time_cost_info: 
|1|0.0028902259946335107|
|2|0.004740147007396445|
|3|0.008479182986775413|
|4|0.01404086998081766|
|8|0.05427642198628746|
|6|0.0709600190166384|
|5|0.022043261007638648|
rand_seed: 524453158


 12%|█████▌                                      | 4/32 [00:00<00:05,  4.99it/s]

...

time_cost_info: 
|1|0.003769953007576987|
|2|0.004799586982699111|
|3|0.009313902002759278|
|4|0.015315561002353206|
|8|0.06110972701571882|
|6|0.17866256699198857|
|7|0.06843936399673112|
rand_seed: 262835631


 97%|█████████████████████████████████████████▋ | 31/32 [00:07<00:00,  4.09it/s]

time_cost_info: 
|1|0.004096807999303564|
|2|0.0056844259961508214|
|3|0.008499172981828451|
|4|0.013857352023478597|
|8|0.06790440698387101|
|6|0.07069491597940214|
|5|0.02191727701574564|
rand_seed: 240490034


100%|███████████████████████████████████████████| 32/32 [00:07<00:00,  4.30it/s]

time_cost_info: 
|1|0.005072025989647955|
|2|0.004054396005813032|
|3|0.010418879013741389|
|4|0.013837263017194346|
|8|0.07538325097993948|
|6|0.059564682975178584|
|7|0.06814941999618895|


In [6]:
ref_count

4902

In [7]:
test_cnt

4672

In [8]:
len(rules)

50

In [2]:
from bds.bbSampleTreeApproxCounting import BranchAndBoundNaive
import time 
import random 

In [6]:
for l in [1]: 
    for k in [6500]: 
        bb = BranchAndBoundNaive(rules, ub, y, lmbd, l=l, k=k) 
        Z = bb.runST()
        print("count " + str(Z))

NameError: name 'rules' is not defined

In [ ]:
all_counts = [] 
for _ in range(100): 
    for l in [1]: 
        for k in [1500]: 
            bb = BranchAndBoundNaive(rules, ub, y, lmbd, l=l, k=k) 
            Z = bb.runST()
            all_counts.append(Z) 
    

In [ ]:
np.mean(all_counts), np.median(all_counts)

In [ ]:
df = pd.DataFrame(
    res_rows,
    columns=['num_rules', 'running_time_approx_mc2', 'running_time_bb', 'estimate_count', 'true_count']
)
df['runtime-factor'] = df['running_time_approx_mc2'] / df['running_time_bb']
df['estimation-rel-diff'] = (df['estimate_count'] - df['true_count']) / df['true_count']

df



In [ ]:
print(df.groupby('num_rules')['running_time_approx_mc2', 'running_time_bb', 'runtime-factor'].mean().to_markdown())

In [ ]:
y

In [3]:
num_rules = 2
rule_random_seeds = randints(num_rules)
rules = [
    Rule.random(i+1, num_pts, random_seed=rule_random_seeds[i])
    for i in range(num_rules)
]
y = bin_random(num_pts)

In [4]:
y = np.array([0, 0, 1, 0, 1], dtype=bool)
from bds.rule import Rule
from bds.utils import mpz_set_bits
from gmpy2 import mpz, mpfr


rules = [
        Rule(
            id=1,
            name="rule-1",
            cardinality=1,
            # truthtable=np.array([0, 1, 0, 1, 0], dtype=bool),
            truthtable=mpz_set_bits(mpz(), [1, 3])
        ),
        Rule(
            id=2,
            name="rule-2",
            cardinality=1,
            # truthtable=np.array([0, 0, 1, 0, 1], dtype=bool),
            truthtable=mpz_set_bits(mpz(), [2, 4])
        ),
        Rule(
            id=3,
            name="rule-3",
            cardinality=1,
            # truthtable=np.array([1, 0, 1, 0, 1], dtype=bool),
            truthtable=mpz_set_bits(mpz(), [0, 2, 4])
        ),
    ]



In [5]:
bb = BranchAndBoundNaive(rules, 10 , y, lmbd, l=1, k=100000) 
Z = bb.runST()

calling!
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
calling!
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
calling!
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
calling!
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
4

In [30]:
2**3 - 1

7

In [28]:
len([{0, 1}, {0}, {0,1,2}, {0,2}, {0,1,3}, {0,3}, {0,1,2,3}, {0,2,3}])

8

In [31]:
Z

8.0